### Práctica de Formatos de datos: CSV y JSON

In [ ]:
# Víctor Fernández Rubio
# David González Jiménez
# Carlos Llames Arribas
# Marta Pastor Puente

#### Ejercicio 1[5 puntos]
Considerar los archivos __Terrazas.csv__ y __Locales.csv__ que contiene un censo de locales y actividades del Ayuntamiento de Madrid, clasificados según su tipo de acceso (puerta de calle o agrupado), situación (abierto, cerrado...) e indicación de la actividad económica ejercida y de las terrazas de hostelería y restauración que aparecen registradas en dicho censo. Observar que la información puede contener alguna inconsistencia.Se pide realizar funciones en Python que permitan realizar las siguientes acciones:

* Usando los dos archivos indicados, se pide crear un nuevo archivo csv denominados __NoTerrazas.csv__ el cual contenga la información de los locales que no son terrazas. Es decir, contendrá la misma información que el archivo __Locales.csv__ pero sin la información de locales que son Terrazas.

* Usando el archivo __Locales.csv__ se pide crear una nuevo archivo csv denominada __Agrupadas.csv__ el cual debe contener una línea por cada barrio conteniendo el número de locales en los que el nombre del vial del edificio en que se encuentran es diferente del nombre del vial de acceso al local. Por ejemplo el local con id 10000401 se encuentra en la Plaza de Isabel II pero el acceso al mismo se realiza desde la Calle Arenal. La información en el archivo debe aparecer en el siguiente orden:

  Código Barrio, Nombre Barrio, Número de empresas cumpliendo la condición
  


* Usando el archivo __Terrazas.csv__ crea un nuevo archivo cvs denominado TopRepeticiones.cvs. Considera el número de veces que se repite el nombre de una terraza (campo rótulo). Se pide crear un archivo csv que contenga los 20 nombres que más se repiten. La información en el archivo debe aparecer en orden descendente con los siguientes campos de información:

  Nombre terraza, Número de repeticiones

Para leer los archivos usa un código como el siguiente:

In [2]:
# NoTerrazas.csv

import csv
import operator

if __name__ == "__main__":
    # We use try-except statements to properly handle errors, as in case the
    # input file does not exist in the directory.
    try:
        # We open the .csv file loading the filds separated by a ; delimiter:
        csv_archivo_locales = open("Locales.csv", encoding="utf8", errors='ignore')
        locales = csv.reader(csv_archivo_locales, delimiter=";")
        csv_archivo_terrazas = open("Terrazas.csv", encoding="utf8", errors='ignore')
        terrazas = csv.reader(csv_archivo_terrazas, delimiter=";")

        # We skip the first line before saving the results into a list for later
        # processing by using the next() statement. The reason why we do not use
        # the line_num function is because we would need to include it in a loop
        # and read file line by line whereas this way, with just two instructions
        # we get the same result:
        next(locales, None)
        lista_locales = list(locales)
        next(terrazas, None)
        lista_terrazas = list(terrazas)

        # When we read the fields from the CSV, they are stored as strings, so we
        # need to explicitely convert the ID to int to be able to sort them:
        for i in lista_locales:
            i[0] = int(i[0])

        for j in lista_terrazas:
            j[0] = int(j[0])

        # We sort the lists taking into account the ID column which is in the
        # first position to get it with the itemergetter function:
        sorted_lista_locales = sorted(lista_locales, key=operator.itemgetter(0), reverse = False)
        sorted_lista_terrazas = sorted(lista_terrazas, key=operator.itemgetter(0), reverse = False)

        # For each entry in lista_terrazas, we check where is its corresponding
        # entry in lista_locales. As they are sorted ascendently, all the entries
        # before the wanted one are included in the result list as they do not
        # appear in lista_terrazas. Moreover, we keep the index counter to just
        # traverse the lista_locales once.
        index = 0
        no_terrazas = []
        for terraza in sorted_lista_terrazas:
            while (terraza[0] > sorted_lista_locales[index][0]):
                no_terrazas.append(sorted_lista_locales[index])
                index += 1

            # It is important to perform this step once we have reached the local
            # entry from the lista_locales, as if not, the next entry from
            # lista_terrazas will be greater than the found one in lista_locales
            # and will wrongly include it in the result list:
            if (terraza[0] == sorted_lista_locales[index][0]):
                index += 1

        # We open the output file to store the data retrieved:
        csvFileObj = open("NoTerrazas.csv", "w")
        csvWriter = csv.writer(csvFileObj, delimiter=";")

        # For each row in the final list, we write it to the CSV output file:
        for row in no_terrazas:
            csvWriter.writerow(row)
            
        print ("The file has been successfully created.")

        # We close the opened CSV file:
        csvFileObj.close()

        # NOTE: we have realized that a few error occurs when comparing the total
        # amount of entries in NoTerrazas.csv with the entries in Locales.csv and
        # Terrazas.csv, as it is expected to have NoTerrazas = Locales - Terrazas

        # However, as the exercice statement indicates that some inconsistency
        # may exist in the files, we assume the error is due to this unsteadiness.

    except IOError:
        print("The file does not exist.")

The file has been successfully created.


In [3]:
# Agrupadas.csv
import csv

if __name__ == "__main__":
    # We use try-except statements to properly handle errors, as in case the
    # input file does not exist in the directory.
    try:
        # We open the .csv file loading the filds separated by a ; delimiter:
        csv_archivo_locales = open("Locales.csv", encoding="utf8", errors='ignore')
        entrada_locales = csv.reader(csv_archivo_locales, delimiter=";")

        # We skip the first line before saving the results into a list for later
        # processing by using the next() statement. The reason why we do not use
        # the line_num function is because we would need to include it in a loop
        # and read file line by line whereas this way, with just two instructions
        # we get the same result:
        next(entrada_locales, None)
        lista_locales = list(entrada_locales)

        # List that will contain the result:
        agrupadas = []
        # Dictionary to keep the relation of neighborhood ID and its name, and
        # the number of locals whose main entry address differs from its official
        # address:
        barrios_dic = {}

        # For each local in the list of locals, we check if it is already in the
        # dictionary to increase its counter. Otherwise, we create a new entry in
        # the dictionary and set the counter to 1:
        for local in lista_locales:
            if local[16] != local[25]:
                entry = local[3] + ',' + local[4]

                if entry in barrios_dic:
                    barrios_dic[entry] += 1
                else:
                    barrios_dic[entry] = 1

        # Now we get the key (neighborhood ID + neighborhood name) and its attached
        # value (counter). To split the key into the two different fields it is
        # containing, we use the split() function and store the fields in a
        # small list also with the counter value:
        for key, value in barrios_dic.items():
            lista_barrios = []
            info_barrio = key.split(',')

            for i in info_barrio:
                lista_barrios.append(i)
            lista_barrios.append(value)

            # Finally, we store the mini-list containing the requested information
            # for each neighborhood in a list of neighborhoods:
            agrupadas.append(lista_barrios)

        # We open the output file to store the data retrieved:
        csvFileObj = open("Agrupadas.csv", "w")
        csvWriter = csv.writer(csvFileObj, delimiter=";")

        # For each row in the final list, we write it to the CSV output file:
        for row in agrupadas:
            csvWriter.writerow(row)
            
        print ("The file has been successfully created.")

        # We close the opened CSV file:
        csvFileObj.close()

    except IOError:
        print("The file does not exist.")

The file has been successfully created.


In [6]:
# TopRepeticiones.csv
import csv
import operator

if __name__ == "__main__":
    # We use try-except statements to properly handle errors, as in case the
    # input file does not exist in the directory.
    try:
        # We open the .csv file loading the filds separated by a ; delimiter:
        csv_archivo_terrazas = open("Terrazas.csv", encoding="utf8", errors='ignore')
        terrazas = csv.reader(csv_archivo_terrazas, delimiter=";")

        # We skip the first line before saving the results into a list for later
        # processing by using the next() statement. The reason why we do not use
        # the line_num function is because we would need to include it in a loop
        # and read file line by line whereas this way, with just two instructions
        # we get the same result:
        next(terrazas, None)
        lista_terrazas = list(terrazas)

        # Dictionary to keep the relation of local banner text and the number of
        # locals whith the same banner name:
        terrazas_dic = {}

        # For each local in the list of locals, we check if it is already in the
        # dictionary to increase its counter. Otherwise, we create a new entry in
        # the dictionary and set the counter to 1:
        for terraza in lista_terrazas:
            entry = terraza[21]

            if entry in terrazas_dic:
                terrazas_dic[entry] += 1
            else:
                terrazas_dic[entry] = 1

        # Now we get the key (local banner text) and its attached value (counter).
        # Then we store both values in a mini-list which will be also store in a
        # list of lists (each of one of those lists containing the values banner
        # text and amount of times it is repeated among all the locals):
        lista_rotulos = []
        for key,value in terrazas_dic.items():
            lista_num_rotulos = []

            lista_num_rotulos.append(key)
            lista_num_rotulos.append(value)

            lista_rotulos.append(lista_num_rotulos)

        # We sort the list taking into account the counting column which is in
        # the second position to get it with the itemergetter function:
        sorted_list_rotulos = sorted(lista_rotulos, key=operator.itemgetter(1), reverse = True)

        # Finally, we store in the result list only the first 20 values, already
        # sorted in descented mode:
        top_repeticiones = []
        for i in range(0, 20):
            top_repeticiones.append(sorted_list_rotulos[i])

        # We open the output file to store the data retrieved:
        csvFileObj = open("TopRepeticiones.csv", "w")
        csvWriter = csv.writer(csvFileObj, delimiter=";")

        # For each row in the final list, we write it to the CSV output file:
        for row in top_repeticiones:
            csvWriter.writerow(row)
            
        print ("The file has been successfully created.")

        # We close the opened CSV file:
        csvFileObj.close()

    except IOError:
        print("The file does not exist.")

The file has been successfully created.


#### Ejercicio 2[5 puntos]
Considerar el archivo __Agenda.json__ que contiene información de actividades de carácter gratuito, para público infantil y adulto, desarrolladas por las siguientes instituciones de titularidad municipal: la red de Bibliotecas Públicas, la Biblioteca Histórica, la Biblioteca Musical Víctor Espinós, la Hemeroteca y la Imprenta. Se presentan las actividades de los próximos 60 dias. Se pide realizar funciones en Python que permitan realizar las siguientes acciones:

* El archivo contiene un campo denominado "audience" que indica el público al que va dirigido. A veces aparece y otras veces no aparece dicho campo. Se quiere procesar el archivo para obtener como resultado dos archivos de texto:el archivo __Niños.txt__ que contendrá la información de los eventos que es exclusivamente para niños y el archivo __Familias.txt__ que contendrá la información de los eventos que está dirigida a niños y familias. En los archivos debe aparecer una línea por cada evento cumpliendo las condiciones indicadas, y mostrando la información siguiente:
  
   Título del evento(campo title), inicio(campo "dtstart"), finalizacion(campo "dtend"), url(campo link)
   
   
* Se desea conocer los eventos más cercanos a un punto que viene especificado por su longitud y latitud. De manera que se quiere crear una función que tomando como entrada una longitud, una latitud y un número real que representa una distancia en kilometros, devuelva como resultado los eventos que se encuentran a una distancia menor o igual a la distancia dada como parámetro. El resultado se mostrará por la pantalla mostrando una línea por cada evento. De cada evento se mostrará:

  Título del evento(campo title), inicio(campo "dtstart"), finalizacion(campo "dtend"), url(campo link)
  
Para calcular la distancia entre dos puntos dadas sus coordenadas se utilizará la siguiente función en Python:

Para leer los archivos usa un código como el siguiente:

In [11]:
# Audience
import json

if __name__ == "__main__":
    # We use try-except statements to properly handle errors, as in case the
    # input file does not exist in the directory.
    try:
        # We open the .json file and properly load it in JSON format using the
        # json module for Python
        doc = open('Agenda.json', encoding="utf8")
        json_doc = json.loads(doc.read())
        lista_eventos = json_doc["@graph"]

        try:
            # We open both the output files to store the data retrieved
            kids = open("Children.txt", "w", encoding="utf8")
            familias = open("Families.txt", "w", encoding="utf8")

            # For each entry in the list of events, we check if the field
            # "audience" corresponds to one of the ones we are looking for. If
            # it is the case, we add it to the correspondent output file:
            for evento in lista_eventos:
                if "audience" in evento:
                    if evento["audience"] == "Niños,Familias":
                        familias.write("Titulo: " + evento["title"] + "; " + "Inicio: " + evento["dtstart"] + "; " + "Final: " + evento["dtend"] + "; " + "URL: " + evento["link"] + "\n")
                    elif evento["audience"] == "Niños":
                        kids.write("Titulo: " + evento["title"] + "; " + "Inicio: " + evento["dtstart"] + "; " + "Final: " + evento["dtend"] + "; " + "URL: " + evento["link"] + "\n")

            print ("The files have been successfully created.")

            # We close every opened file:
            kids.close()
            familias.close()
            doc.close()

        except IOError:
            print ("The files could not be opened.")

    except IOError:
        print("The file does not exist.")

The files have been successfully created.


In [2]:
# Distance
import json
import math

def haversine(lat1, lon1, lat2, lon2):
    rad=math.pi/180
    dlat= lat2 - lat1
    dlon= lon2 - lon1
    R=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia


def eventos_cercanos(latitud, longitud, max_distancia, json_doc):
    lista_eventos = json_doc["@graph"]

    # For every event in the list of events, we obtain its latitude and longitude
    # and calculate the distance in km using the given function.
    for loc in lista_eventos:
        if "location" in loc:
            lat = loc["location"]["latitude"]
            lon = loc["location"]["longitude"]
            dis = haversine(float(latitud), float(longitud), lat ,lon)

            # If the distance calculated is lower than our maximun distance wanted
            # then the activity is shown in the standard output:
            if dis <= float(max_distancia):
                print ("Titulo: " + loc["title"] + "; " + "Inicio: " + loc["dtstart"] + "; " + "Final: " + loc["dtend"] + "; " + "URL: " + loc["link"] + "\n")


if __name__ == "__main__":
    # We use try-except statements to properly handle errors, as in case the
    # input file does not exist in the directory.
    try:
        # We open the .json file and properly load it in JSON format using the
        # json module for Python
        doc = open('Agenda.json', encoding="utf8")
        json_doc = json.loads(doc.read())

        lista_eventos = json_doc["@graph"]

        # We introduce the desired values to calculate from:
        miLat = input ("Mi latitud: \n")
        miLon = input ("Mi longitud: \n")
        distMax = input ("Distancia maxima que recorrer: \n")

        eventos_cercanos(miLat, miLon, distMax, json_doc)

        # We close the opened file:
        doc.close()

    except IOError:
        print("The file does not exist.")

Mi latitud: 
40
Mi longitud: 
-3
Distancia maxima que recorrer: 
80
Titulo: A medias; Inicio: 2017-10-22 19:00:00.0; Final: 2017-10-22 23:59:00.0; URL: http://www.madrid.es/sites/v/index.jsp?vgnextchannel=1ccd566813946010VgnVCM100000dc0ca8c0RCRD&vgnextoid=ad3b9c7bd059e510VgnVCM1000001d4a900aRCRD

Titulo: American Cultural Bites; Inicio: 2017-10-27 19:00:00.0; Final: 2017-10-27 23:59:00.0; URL: http://www.madrid.es/sites/v/index.jsp?vgnextchannel=1ccd566813946010VgnVCM100000dc0ca8c0RCRD&vgnextoid=c34634b9e7cde510VgnVCM2000001f4a900aRCRD

Titulo: Bambi; Inicio: 2017-10-30 18:00:00.0; Final: 2017-10-30 23:59:00.0; URL: http://www.madrid.es/sites/v/index.jsp?vgnextchannel=1ccd566813946010VgnVCM100000dc0ca8c0RCRD&vgnextoid=c86f1fcc397ce510VgnVCM2000001f4a900aRCRD

Titulo: Bebecuentos con Carlos Alba; Inicio: 2017-12-04 17:30:00.0; Final: 2017-12-04 23:59:00.0; URL: http://www.madrid.es/sites/v/index.jsp?vgnextchannel=1ccd566813946010VgnVCM100000dc0ca8c0RCRD&vgnextoid=c1a0121bd69be510VgnVCM1

#### Normas de entrega

* Fecha tope de entrega: 27/10/2017
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre FormatosI_GrupoX donde X será el número de grupo correspondiente.